In [ ]:
from dotenv import load_dotenv
import os
from azure.identity import DefaultAzureCredential,get_bearer_token_provider
from langchain_openai import AzureChatOpenAI
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from typing  import Literal,Sequence,Annotated,TypedDict
import operator
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode,tools_condition
from langgraph.checkpoint.memory import MemorySaver


load_dotenv()



os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")  # Update to your API key
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_APIKEY")


token_provider= get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
model= AzureChatOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint="https://azopenai-langchain.openai.azure.com/",
    azure_ad_token_provider= token_provider,
    azure_deployment= "gpt-4o-mini"
)
model.invoke("HI").content


In [ ]:
@tool
def multiply(a:int,b:int):
    """ This tool will multiply the given two imput and return the response"""
    return a*b


@tool
def search(query:str):
    """search the web for a query and return the respone"""
    web_results= TavilySearchResults(max_results=1).invoke(query)
    return f"Result for {query} is ::  {web_results}"
tools = [multiply,search]
llm_with_tools = model.bind_tools(tools=tools)

class AgentState(TypedDict):
    messages : Annotated[Sequence[BaseMessage],operator.add]

def ai_assistant(state: AgentState):
   print("in assitant")
   response=  llm_with_tools.invoke(state["messages"])
   print(response)
   return {"messages":[response]}


In [ ]:

memory= MemorySaver()
graph= StateGraph(AgentState)
graph.add_node("ai_assistant",ai_assistant)
graph.add_node("tools",ToolNode([multiply,search]))
graph.add_conditional_edges("ai_assistant",tools_condition)
graph.add_edge("tools","ai_assistant")
graph.add_edge(START, "ai_assistant")
config= {"configurable" : {"thread_id" :"1"}}
compiledgraph=graph.compile(checkpointer=memory,interrupt_before=["tools"])
compiledgraph

In [ ]:
from langchain_core.messages import HumanMessage,AIMessage,ToolMessage
new_message=[
    ToolMessage
]
response=compiledgraph.invoke({"messages":[HumanMessage("What is the current gdp of the china?")]},config=config)

In [ ]:
snapshot=compiledgraph.get_state(config)

In [ ]:
snapshot.next
tool_details=snapshot.values["messages"][-1].tool_calls
tool_details

In [ ]:
if tool_details[0]["name"]== "search":
    user_input=input(prompt=f"[yes/no] do you want to continue with {tool_details[0]['name']}?").lower()
    if user_input=="no":
        print("web tool discarded")
        raise Exception("Web tool discarded by the user.")
    else:
        response=compiledgraph.invoke(None,config)
        print(response)
else:
    response=compiledgraph.invoke(None,config)
    print(response)
    

In [ ]:
compiledgraph.get_state(config)
